# **Prova Final: Computação Científica II - 2023**
##### **Data**: 30/11/2023 - 03/12/2023
##### **Alune**: Igor Lima Strozzi
##### **DRE**: 112040449

## Questão 2


Temos de resolver o seguinte BVP:

$$
\epsilon u'' + 2 (1 - x^2) u = 1 \tag{1.1}
$$

sob as condições de contorno:

$$
u(-1) = u(1) = 0 \tag{1.2}
$$.

O *ansatz* é dado, sendo as funções base $\phi_j(x) = \sin(j \pi x)$ e a aproximação da solução é uma combinação linear:
$$
u(x) \approx u_h(x) = \sum_j^N c_j \phi_j(x) \tag{2}
$$

Observemos que: 
* As funções base respeitam as condições de contorno;
* Trataremos o domínio como sendo $D = (-1, 1)$;
* $N$ é a dimensão do espaço de funções usado.

A idéia geral de resolução é: 
1. Aplicar $u_h$ a $(1.1)$;
2. Usar o método de Galerkin para encontrar um sistema linear de equações;
3. Resolver o sistema linear encontrado, obtendo os coeficientes $c_j$ das funções da base

O método de Galerkin exige que, para que $u_h(x)$ resolva a EDO, temos que garantir que
$$\int_{-1}^{1} r(x)\phi_i(x) \, dx = 0$$ 
com $r(x) = \epsilon u_h'' + 2(1 - x^2)u_h + u_h^2 - 1$, obtido substituindo $u_h$ no na EDO.

Agora, reescrevendo $u_h$ como combinação linear das funções da base, chegamos a
$$\int_{-1}^{1} \left( \epsilon \left( \sum_j^{N} c_j \phi_j'' \right) + 2(1 - x^2)\left( \sum_j^{N} c_j \phi_j \right) + \left( \sum_j^{N} c_j \phi_j \right)^2 - 1 \right) \phi_i \, dx = 0$$


Podemos observar um número de coisas. Em primeiro lugar, a paridade de $\phi_j = \sin(j \pi x)$, considerada juntamente à simetria do domínio de integração, e o uso das condições de contorno nos resultados das integrações por partes ajudam a simplificar largamente as integrais. Por exemplo, observe que $\left( \sum_j^{N} c_j \phi_j \right)$ é soma de funções ímpar, então é ímpar; $\left( \sum_j^{N} c_j \phi_j \right)^2$, no entanto, é par. $\phi_i$, por sua vez, é impar. Logo $\left( \sum_j^{N} c_j \phi_j \right)^2\phi_i$ é uma função ímpar, e sua integral sobre $D$ é nula. O mesmo vale pra integral do último termo, $-\phi_i$. Com isso, ficamos apenas com:
$$\int_{-1}^{1} r(x)\phi_i(x) \, dx = \int_{-1}^{1}  \sum_{j}^{N} c_j \left( \epsilon \phi_j'' + 2(1 - x^2)\phi_j \right) \phi_i \, dx = 0$$

Como a integral independe de $j$, podemos reescrever a equação acima como (chamando $(1 - x^2)$ de $\alpha$):
$$
  \sum_{j}^{N} c_j \int_{-1}^{1}\left( \epsilon \phi_j'' + 2\alpha\phi_j \right) \phi_i \, dx = 0
$$
mais ainda, como 
$$\sum_{j}^{N} c_j \left( \epsilon \int_{-1}^{1} \phi_j'' \phi_i \, dx + 2 \int_{-1}^{1} \alpha\phi_j \phi_i \, dx \right) = 0$$

Consideremos a primeira integral. Integrando por partes e aplicando a condição de contorno, obtemos $-\epsilon \int_{-1}^{1} \phi_j' \phi_i' \, dx$, e ficamos com
$$\sum_{j}^{N} c_j \left( -\epsilon \int_{-1}^{1} \phi_j' \phi_i' \, dx + 2 \int_{-1}^{1} \alpha\phi_j \phi_i \, dx \right) = 0$$

Bom, há aqui, agora, um grande problema. Podemos observar, a partir da equação $(4.70)$ do livro (p. 95), que pra que tenhamos um sistema linear bem definido, temos que ter um termo que não dependa de $u_h$, ou, mais especificamente, de $j$, dependendo só da $\phi_i$ multiplicada e de $x$. Com o que temos, o sistema que encontramos é
$$
\mathrm{A}_{ij} = -\epsilon \int_{-1}^{1} \phi_j' \phi_i' \, dx + 2 \int_{-1}^{1} \alpha\phi_j \phi_i \, dx \\
\mathbf{c} = [c_j]^T \\
\mathbf{f} = [f_i]^T \\
f_i = -\int_{-1}^{1} \phi_i \, dx
$$


O que nos dá o sistema $\mathrm{A}\mathbf{c} = \mathbf{f}$. Mas, como observamos, $f_i = -\int_{-1}^{1} \sin(i \pi x) \, dx = 0$. De fato, calculando a integral indefinida obtemos $\frac{-1}{i \pi} \cos(i \pi x)$, o que implica
$$
-\int_{-1}^{1} \sin(i \pi x) , dx =-\left[\frac{-1}{i \pi} \cos(i \pi x)\right]_{-1}^1 = - \left(\frac{-1}{i \pi} \cos(i \pi x) \bigg|_1 - \frac{-1}{i \pi} \cos(i \pi x) \bigg|_{-1} \right)\\
= - \left(\frac{-1}{i \pi} (-1) - \frac{-1}{i \pi} (-1)\right) = -\left(\frac{1}{i \pi} - \frac{1}{i \pi} \right)= -0
$$

Logo $\mathrm{A}\mathbf{c} = \mathbf{f} = \mathbf{0}$, que é um sistema homogêneo. Então não tem solução não-trivial; dependendo da $\mathrm{A}$, vai ter infinitas soluções não triviais ou uma única, $\mathrm{0}$. De qualquer forma, façamos um código pra resolvê-lo, integrando numericamente os coeficientes de $\mathrm{A}$.

In [1]:
import numpy as np
from scipy.integrate import quad
from scipy.linalg import solve
from scipy.linalg import det

# Define the basis function and its derivative
def phi(i, x):
    return np.sin(i * np.pi * x)

def dphi(i, x):
    return i * np.pi * np.cos(i * np.pi * x)


# Integral calculations
def compute_Aij(i, j, epsilon):
    integral1, _ = quad(lambda x: -epsilon * dphi(i, x) * dphi(j, x), -1, 1)
    integral2, _ = quad(lambda x: 2 * (1-x**2) * phi(i, x) * phi(j, x), -1, 1)
    return integral1 + integral2

def compute_fi(i):
    integral, _ = quad(lambda x: -phi(i, x), -1, 1)
    return integral

# Parameters
epsilon = 1  # This should be set to a specific value
N = 3         # Number of basis functions (dimension of the system)

# Constructing the matrix A and vector f
A = np.zeros((N, N))
f = np.zeros(N)

for i in range(1, N + 1):
    f[i - 1] = compute_fi(i)
    for j in range(1, N + 1):
        A[i - 1, j - 1] = compute_Aij(i, j, epsilon)
print(f"A = {A} \n")
det_A = det(A)

if det_A == 0:
    print(f"The determinant of A is {det_A}. Hence either no or infinite solutions.")
else:
    print(f"The determinant of A is non-zero and equals {det_A}. Hence no non-trivial solutions and the solution is 0.")

# Solve the linear system Ac = f
c = solve(A, f)
c



A = [[-8.43494988e+00  3.60253097e-01 -7.59908877e-02]
 [ 3.60253097e-01 -3.81197540e+01  3.89073345e-01]
 [-7.59908877e-02  3.89073345e-01 -8.74818484e+01]] 

The determinant of A is non-zero and equals -28115.928028397608. Hence no non-trivial solutions and the solution is 0.


array([-0., -0., -0.])

Como podemos ver no código acima, o determinante de $\mathrm{A}$ de fato é não nulo (ele varia com $\epsilon$ e $N$, e é uma questão se pra algum ele é zero, mas eu testei pra valores críticos e $\det A$ parece, efetivamente, sempre não nulo). Nesse caso, a solução é única: $\mathbf{c} = \mathbf{0}$, como confirmado no snippet acima.